# Brazil Cellular Data Cleanup

This notebook merges the raw cellular data for Brazil into a format that can be used in the models.
This is expected to be a one time tranformation, but if more cellular data for Brazil becomes availible, the following can be used to transform it again.
The current raw dataset for this, can be found [here](https://drive.google.com/drive/folders/1dulyARHZl4gvYYUmk6hvMebonvkGot8U?usp=share_link).

In [ ]:
from glob import glob
from os.path import join, getctime
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

WORKSPACE_SOURCE = '../parameter_workspace/cellular_br/'
WORKSPACE_TARGET = '../sample_workspace/brazil/'

def get_sort_files(path, extension):
    list_of_files = []
    for file in glob(join(path,f'*{extension}')):
        list_of_files.append((getctime(file), file))
    return [file for _, file in sorted(list_of_files)]

def drop_duplicates(frame, primary_key):
    return frame.drop_duplicates(subset=primary_key, keep="last")

# load in all the data, and de-duplicate first

primary_key = 'NumEstacao'
cell_towers_raw = pd.DataFrame()
table_files = get_sort_files(WORKSPACE_SOURCE, 'csv')
for f in tqdm(table_files):
    df = pd.read_csv(f, encoding = "iso-8859-1")
    df = drop_duplicates(df, primary_key)
    cell_towers_raw = pd.concat([cell_towers_raw, df])
cell_towers_raw = drop_duplicates(cell_towers_raw, primary_key)

In [ ]:
cell_towers_raw

In [ ]:
# re-name and drop irrelevant columns
cell_towers_raw = cell_towers_raw.rename(columns={'NumEstacao': 'Site ID',
                                                  'AlturaAntena': 'Tower Height',
                                                  'Tecnologia': 'Technology'})

cell_towers_raw['Ownership of site'] = 'Unknown'
cell_towers_raw['Indoor /outdoor'] = 'Outdoor'
cell_towers_raw['Technology'] = 'LTE'
cell_towers_raw = cell_towers_raw[['Site ID', 'Ownership of site',
                                   'Indoor /outdoor', 'Latitude', 'Longitude',
                                   'Tower Height', 'Technology']]
cell_towers_raw = cell_towers_raw.dropna(subset=['Latitude', 'Longitude'])
cell_towers_raw.to_csv(join(WORKSPACE_TARGET, 'cellular.csv'))